# Compito Esercitazione 6
* Import E6
* Data Loader E6
* **ResNet50 vs InceptionV3 Fine Tuning**
* **Data Augmentation**
* **Relazione**

# Import E6

In [1]:
# (Solo per Google Colab)
%tensorflow_version 2.x
!pip install -q keras==2.3.0

     |████████████████████████████████| 378kB 6.7MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 


In [2]:
import tensorflow.keras
from tensorflow.keras.applications import mobilenet_v2, ResNet50, InceptionV3
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

import numpy as np
from time import time

# Data Loader E6

In [3]:
t0 = time()
# Download del dataset tramite wget
!wget https://www.dropbox.com/s/drwy7fq5svwv78p/101_ObjectCategories_split.tar

--2020-08-29 13:29:29--  https://www.dropbox.com/s/drwy7fq5svwv78p/101_ObjectCategories_split.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/drwy7fq5svwv78p/101_ObjectCategories_split.tar [following]
--2020-08-29 13:29:29--  https://www.dropbox.com/s/raw/drwy7fq5svwv78p/101_ObjectCategories_split.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0f28c7157d6814fe3171268675.dl.dropboxusercontent.com/cd/0/inline/A-bQmGqY5nHTNW-SAf_1GXBexX4ZcFWaX1DLR-vMd62AWBDA9NOOY0fFg_SsaJ2b-HumR-7OZJqEoFQxKo11iCS5AiUkSVv-wuZ8KznkAKR4p-wjpMYg_lddCCD8hmcBmqs/file# [following]
--2020-08-29 13:29:29--  https://uc0f28c7157d6814fe3171268675.dl.dropboxusercontent.com/cd/0/inline/A-bQmGqY5nHTNW-SAf_1GXBexX4ZcFWaX1DLR-vMd62AWBDA9NOOY0fFg

In [4]:
# Estrazione file e rimozione archivio
import tarfile
tar = tarfile.open('101_ObjectCategories_split.tar')
tar.extractall()
tar.close()
!rm 101_ObjectCategories_split.tar
base_path = '101_ObjectCategories_split/'

In [13]:
# Trasformazioni da applicare
train_processing_res = kimage.ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.resnet.preprocess_input)
train_processing_inc = kimage.ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.inception_v3.preprocess_input)

# Informazioni su dove reperire i dati e come dividerli
train_generator_res = train_processing_res.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)
train_generator_inc = train_processing_inc.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

# test
test_processing_res = kimage.ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.resnet.preprocess_input)
test_generator_res = test_processing_res.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)
test_processing_inc = kimage.ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.inception_v3.preprocess_input)
test_generator_inc = test_processing_inc.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 4600 images belonging to 102 classes.
Found 4600 images belonging to 102 classes.
Found 4544 images belonging to 102 classes.
Found 4544 images belonging to 102 classes.


# ResNet50 vs InceptionV3 Fine Tuning

## ResNet50
ResNet50 è una CNN a 50 strati. Apparentemente rendere i modelli più profondi è il metodo basilare per migliorare la performance o affrontare problemi più complessi, ma come per i percettroni agli albori degli studi nel campo, è difficile allenare modelli all'aumentare degli strati, in particolare perché i gradienti nella backpropagation "svaniscono" ed i pesi negli strati iniziali vengono modificati troppo poco. 
ResNet è organizzata a blocchi in cui viene approssimata non la funzione obiettivo ideale, bensì quella residuale, al netto degli input del blocco. In questo modo ogni blocco ha dei gradienti utili.

## InceptionV3
InceptionV3 è una versione della rete Inception, in cui moduli analoghi locali con struttura interna parallela, e di comprovata efficacia, vengono riprodotti in serie. Dei classificatori ausiliari biforcano lungo la serie, in modo da iniettare gradienti significativi anche per i moduli a monte della serie stessa.

In [14]:
# Modelli di base
res_base_net = ResNet50(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling='avg')
inc_base_net = InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling='avg')

In [15]:
# Congelamento del modello di base (solo per fine-tuning)
for modello in (res_base_net, inc_base_net):
  for layer in modello.layers:
    layer.trainable = False

# Output del modello di base (Res, poi Inc)
x = res_base_net.output
# Nuovo livello fully-connected intermedio + ReLU
x = Dense(1024, activation='relu')(x)
# Nuovo livello fully-connected finale + softmax
pred_res = Dense(102, activation='softmax')(x)
# Inc
y = inc_base_net.output
y = Dense(1024, activation='relu')(y)
pred_inc = Dense(102, activation='softmax')(y)

# Modelli specializzato
resnet = Model(inputs=res_base_net.input, outputs=pred_res)
incnet = Model(inputs=inc_base_net.input, outputs=pred_inc)

In [16]:
# Compila il modello per l'addestramento
resnet.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
            optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

incnet.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
            optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

In [21]:
resnet.fit(train_generator_res, epochs=2, validation_data=test_generator_res, verbose=1)

Epoch 1/2
144/144 [==============================] - 68s 470ms/step - loss: 1.3302 - accuracy: 0.7213 - val_loss: 0.6482 - val_accuracy: 0.8294
Epoch 2/2
144/144 [==============================] - 67s 462ms/step - loss: 0.2800 - accuracy: 0.9204 - val_loss: 0.4457 - val_accuracy: 0.8805


In [22]:
incnet.fit(train_generator_inc, epochs=2, validation_data=test_generator_inc, verbose=1)

Epoch 1/2
144/144 [==============================] - 49s 337ms/step - loss: 0.3850 - accuracy: 0.9022 - val_loss: 0.6479 - val_accuracy: 0.8565
Epoch 2/2
144/144 [==============================] - 47s 327ms/step - loss: 0.2023 - accuracy: 0.9411 - val_loss: 0.5324 - val_accuracy: 0.8754


# Data Augmentation

La Data Augmentation è l'operazione che aumenta i samples di training applicando diverse trasformazioni ai dati originali, in modo da sopperire al numero relativamente esiguo e soprattutto alla varianza ridotta rispetto al contesto reale o di test in cui si vuole far funzionare il modello. Rotazioni, trasformazioni speculari, variazioni luminose e cromatiche servono a rendere la variabilità del training set (un sampling ridotto e anche distorto della realtà) più simile a quella del "mondo esterno" del deployment di modello.

In [19]:
# preprocessing con augmentation
data_aug_res = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.67, 1.33],
    zoom_range=[0.67, 1.33],
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=tensorflow.keras.applications.resnet.preprocess_input,
    validation_split=0.0
)
data_aug_inc = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.67, 1.33],
    zoom_range=[0.67, 1.33],
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=tensorflow.keras.applications.inception_v3.preprocess_input,
    validation_split=0.0
)

# train
train_generator_aug_res = data_aug_res.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)
train_generator_aug_inc = data_aug_inc.flow_from_directory(
        directory=base_path+'train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        seed=1
)

# test
test_generator_aug_res = data_aug_res.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)
test_generator_aug_inc = data_aug_inc.flow_from_directory(
        directory=base_path+'test',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 4600 images belonging to 102 classes.
Found 4600 images belonging to 102 classes.
Found 4544 images belonging to 102 classes.
Found 4544 images belonging to 102 classes.


In [32]:
# Ricompila il modello per l'addestramento
resnet.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
            optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

incnet.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
            optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

In [33]:
resnet.fit(train_generator_aug_res, epochs=2, validation_data=test_generator_aug_res, verbose=1)

Epoch 1/2
144/144 [==============================] - 165s 1s/step - loss: 0.3742 - accuracy: 0.9080 - val_loss: 1.1946 - val_accuracy: 0.8030
Epoch 2/2
144/144 [==============================] - 164s 1s/step - loss: 0.2964 - accuracy: 0.9207 - val_loss: 1.2765 - val_accuracy: 0.8105


In [34]:
incnet.fit(train_generator_aug_inc, epochs=2, validation_data=test_generator_aug_inc, verbose=1)

Epoch 1/2
144/144 [==============================] - 157s 1s/step - loss: 0.7729 - accuracy: 0.8089 - val_loss: 1.1534 - val_accuracy: 0.7531
Epoch 2/2
144/144 [==============================] - 156s 1s/step - loss: 0.6624 - accuracy: 0.8276 - val_loss: 1.0680 - val_accuracy: 0.7663


Si nota un apprendimento possibilmente rallentato, ma una generalizzazione meno distante dalla performance di training. Vale la pena aumentare il numero di epoche di training ed esaminare i risultati di validation quando l'adattamento al training set sembra simile a quanto visto prima della data augmentation

In [35]:
resnet.fit(train_generator_aug_res, epochs=10, validation_data=test_generator_aug_res, verbose=1)

Epoch 1/10
144/144 [==============================] - 166s 1s/step - loss: 0.2733 - accuracy: 0.9324 - val_loss: 1.2263 - val_accuracy: 0.8063
Epoch 2/10
144/144 [==============================] - 164s 1s/step - loss: 0.2825 - accuracy: 0.9326 - val_loss: 1.2551 - val_accuracy: 0.8072
Epoch 3/10
144/144 [==============================] - 165s 1s/step - loss: 0.3076 - accuracy: 0.9291 - val_loss: 1.3963 - val_accuracy: 0.8050
Epoch 4/10
144/144 [==============================] - 163s 1s/step - loss: 0.2655 - accuracy: 0.9361 - val_loss: 1.6155 - val_accuracy: 0.7980
Epoch 5/10
144/144 [==============================] - 166s 1s/step - loss: 0.3069 - accuracy: 0.9324 - val_loss: 1.3697 - val_accuracy: 0.8160
Epoch 6/10
144/144 [==============================] - 164s 1s/step - loss: 0.2789 - accuracy: 0.9339 - val_loss: 1.4803 - val_accuracy: 0.8193
Epoch 7/10
144/144 [==============================] - 164s 1s/step - loss: 0.3189 - accuracy: 0.9339 - val_loss: 1.5926 - val_accuracy: 0.8046

In [36]:
incnet.fit(train_generator_aug_inc, epochs=10, validation_data=test_generator_aug_inc, verbose=1)

Epoch 1/10
144/144 [==============================] - 155s 1s/step - loss: 0.6794 - accuracy: 0.8220 - val_loss: 1.2815 - val_accuracy: 0.7458
Epoch 2/10
144/144 [==============================] - 155s 1s/step - loss: 0.6684 - accuracy: 0.8304 - val_loss: 1.1007 - val_accuracy: 0.7487
Epoch 3/10
144/144 [==============================] - 155s 1s/step - loss: 0.6430 - accuracy: 0.8361 - val_loss: 1.1067 - val_accuracy: 0.7641
Epoch 4/10
144/144 [==============================] - 155s 1s/step - loss: 0.6709 - accuracy: 0.8293 - val_loss: 1.1916 - val_accuracy: 0.7500
Epoch 5/10
144/144 [==============================] - 155s 1s/step - loss: 0.6471 - accuracy: 0.8311 - val_loss: 1.1568 - val_accuracy: 0.7691
Epoch 6/10
144/144 [==============================] - 155s 1s/step - loss: 0.5987 - accuracy: 0.8448 - val_loss: 1.2660 - val_accuracy: 0.7577
Epoch 7/10
144/144 [==============================] - 155s 1s/step - loss: 0.6408 - accuracy: 0.8352 - val_loss: 1.1653 - val_accuracy: 0.7711

# Relazione

Le due architetture selezionate come base per un classificatore denso, ResNet50 ed InceptionV3, sono piuttosto differenti per dimensioni e struttura, dando luogo a risultati diversi sul validation set. InceptionV3, seppur più complesso, non outperforma ResNet50 a parità di MLP a valle e dataset su cui sono stati allenati i pesi.
La data augmentation tentata ha compreso la rotazione, l'inversione speculare, la traslazione, lo zoom e i cambi di luminosità per i sample. L'insieme di trasformazioni è probabilmente difficile da gestire per il training dei layer densi, ed i risultati potrebbero migliorare permettendo anche ai pesi dei layer trasferiti (sempre inizializzati con i valori allenati su ImageNet) di variare.